'C:\\Users\\noklam.chan\\Documents\\GitHub\\KaggleHousePricePrediction'

In [5]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import StratifiedKFold, KFold
import xgboost
from sklearn.grid_search import ParameterGrid
from sklearn.metrics import mean_squared_error

CLASS = False  # Whether classification or regression
SCORE_MIN = True  # Optimizing score through minimum
k = 5  # Number of folds
best_score = 10
best_params = None
best_iter = None

train_name = 'input/train.csv'
test_name = 'input/test.csv'
#submission_name = '/input/sample_submission.csv'


# Read files
train = pd.DataFrame.from_csv(train_name)
train = train.fillna(-1)
test = pd.DataFrame.from_csv(test_name)
test = test.fillna(-1)
#submission = pd.DataFrame.from_csv(submission_name)
# Extract target
target = train['SalePrice']
del train['SalePrice']

# Label nominal variables to numbers
columns = train.columns.values
nom_numeric_cols = ['MSSubClass']
dummy_train = []
dummy_test = []
for col in columns:
    # Only works for nominal data without a lot of factors
    if train[col].dtype.name == 'object' or col in nom_numeric_cols:
        dummy_train.append(pd.get_dummies(train[col].values.astype(str), col))
        dummy_train[-1].index = train.index
        dummy_test.append(pd.get_dummies(test[col].values.astype(str), col))
        dummy_test[-1].index = test.index
        del train[col]
        del test[col]
train = pd.concat([train] + dummy_train, axis=1)
test = pd.concat([test] + dummy_test, axis=1)

# Use only common columns
columns = []
for col_a in train.columns.values:
    if col_a in test.columns.values:
        columns.append(col_a)
train = train[columns]
test = test[columns]

# CV
train = np.array(train)
target = np.log(np.array(target))  # Changes to Log
test = np.array(test)
print(train.shape, test.shape)

if CLASS:
    kfold = StratifiedKFold(target, k)
else:
    kfold = KFold(train.shape[0], k)

early_stopping = 50

param_grid = [
              {'silent': [1],
               'nthread': [2],
               'eval_metric': ['rmse'],
               'eta': [0.03],
               'objective': ['reg:linear'],
               'max_depth': [5, 7],
               'num_round': [1000],
               'subsample': [0.2, 0.4, 0.6],
               'colsample_bytree': [0.3, 0.5, 0.7],
               }
              ]

# Hyperparmeter grid optimization
for params in ParameterGrid(param_grid):
    print(params)
    # Determine best n_rounds
    xgboost_rounds = []
    for train_index, test_index in kfold:
        X_train, X_test = train[train_index], train[test_index]
        y_train, y_test = target[train_index], target[test_index]

        xg_train = xgboost.DMatrix(X_train, label=y_train)
        xg_test = xgboost.DMatrix(X_test, label=y_test)

        watchlist = [(xg_train, 'train'), (xg_test, 'test')]

        num_round = params['num_round']
        xgclassifier = xgboost.train(params, xg_train, num_round,
                                     watchlist,
                                     early_stopping_rounds=early_stopping);
        xgboost_rounds.append(xgclassifier.best_iteration)

    num_round = int(np.mean(xgboost_rounds))
    print('The best n_rounds is %d' % num_round)
    # Solve CV
    rmsle_score = []
    for cv_train_index, cv_test_index in kfold:
        X_train, X_test = train[cv_train_index, :], train[cv_test_index, :]
        y_train, y_test = target[cv_train_index], target[cv_test_index]

        # train machine learning
        xg_train = xgboost.DMatrix(X_train, label=y_train)
        xg_test = xgboost.DMatrix(X_test, label=y_test)

        watchlist = [(xg_train, 'train'), (xg_test, 'test')]

        xgclassifier = xgboost.train(params, xg_train, num_round);

        # predict
        predicted_results = xgclassifier.predict(xg_test)
        rmsle_score.append(np.sqrt(mean_squared_error(y_test, predicted_results)))

    if SCORE_MIN:
        if best_score > np.mean(rmsle_score):
            print(np.mean(rmsle_score))
            print('new best')
            best_score = np.mean(rmsle_score)
            best_params = params
            best_iter = num_round
    else:
        if best_score < np.mean(rmsle_score):
            print(np.mean(rmsle_score))
            print('new best')
            best_score = np.mean(rmsle_score)
            best_params = params
            best_iter = num_round

# Solution using best parameters
print('best params: %s' % best_params)
print('best score: %f' % best_score)
xg_train = xgboost.DMatrix(train, label=target)
xg_test = xgboost.DMatrix(test)
watchlist = [(xg_train, 'train')]
num_round = best_iter  # already int
xgclassifier = xgboost.train(best_params, xg_train, num_round, watchlist);


(1460, 299) (1459, 299)
{'objective': 'reg:linear', 'colsample_bytree': 0.3, 'eval_metric': 'rmse', 'max_depth': 5, 'silent': 1, 'nthread': 2, 'subsample': 0.2, 'eta': 0.03, 'num_round': 1000}
[0]	train-rmse:11.1852	test-rmse:11.1979
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 50 rounds.
[1]	train-rmse:10.8531	test-rmse:10.8657
[2]	train-rmse:10.5289	test-rmse:10.5416
[3]	train-rmse:10.2136	test-rmse:10.2262
[4]	train-rmse:9.90947	test-rmse:9.92207
[5]	train-rmse:9.6152	test-rmse:9.62778
[6]	train-rmse:9.32729	test-rmse:9.33985
[7]	train-rmse:9.04976	test-rmse:9.0623
[8]	train-rmse:8.78082	test-rmse:8.79334
[9]	train-rmse:8.5197	test-rmse:8.5322
[10]	train-rmse:8.26381	test-rmse:8.27628
[11]	train-rmse:8.01557	test-rmse:8.02802
[12]	train-rmse:7.77655	test-rmse:7.78897
[13]	train-rmse:7.54346	test-rmse:7.55585
[14]	train-rmse:7.31853	test-rmse:7.3309
[15]	train-rmse:7.10094	test-rmse:7.11328
[16]	tr

[182]	train-rmse:0.12349	test-rmse:0.139049
[183]	train-rmse:0.12255	test-rmse:0.138231
[184]	train-rmse:0.121506	test-rmse:0.137272
[185]	train-rmse:0.120475	test-rmse:0.136411
[186]	train-rmse:0.119665	test-rmse:0.135712
[187]	train-rmse:0.118929	test-rmse:0.134989
[188]	train-rmse:0.118193	test-rmse:0.134395
[189]	train-rmse:0.117534	test-rmse:0.133603
[190]	train-rmse:0.116855	test-rmse:0.132983
[191]	train-rmse:0.116093	test-rmse:0.132309
[192]	train-rmse:0.115391	test-rmse:0.131549
[193]	train-rmse:0.114575	test-rmse:0.13071
[194]	train-rmse:0.113753	test-rmse:0.130082
[195]	train-rmse:0.113043	test-rmse:0.129387
[196]	train-rmse:0.112622	test-rmse:0.12887
[197]	train-rmse:0.112079	test-rmse:0.128369
[198]	train-rmse:0.111365	test-rmse:0.127622
[199]	train-rmse:0.110958	test-rmse:0.127312
[200]	train-rmse:0.110384	test-rmse:0.12693
[201]	train-rmse:0.109859	test-rmse:0.126545
[202]	train-rmse:0.109287	test-rmse:0.125911
[203]	train-rmse:0.108888	test-rmse:0.125467
[204]	train-rms

[365]	train-rmse:0.080422	test-rmse:0.110417
[366]	train-rmse:0.080344	test-rmse:0.110353
[367]	train-rmse:0.080301	test-rmse:0.110274
[368]	train-rmse:0.080205	test-rmse:0.110359
[369]	train-rmse:0.080138	test-rmse:0.110377
[370]	train-rmse:0.07993	test-rmse:0.110409
[371]	train-rmse:0.079795	test-rmse:0.110421
[372]	train-rmse:0.079681	test-rmse:0.110073
[373]	train-rmse:0.079596	test-rmse:0.110112
[374]	train-rmse:0.079431	test-rmse:0.110025
[375]	train-rmse:0.079305	test-rmse:0.109939
[376]	train-rmse:0.079163	test-rmse:0.10989
[377]	train-rmse:0.079077	test-rmse:0.109969
[378]	train-rmse:0.078901	test-rmse:0.109891
[379]	train-rmse:0.078828	test-rmse:0.109886
[380]	train-rmse:0.078739	test-rmse:0.109803
[381]	train-rmse:0.078611	test-rmse:0.109755
[382]	train-rmse:0.078486	test-rmse:0.109735
[383]	train-rmse:0.078357	test-rmse:0.109712
[384]	train-rmse:0.078276	test-rmse:0.109795
[385]	train-rmse:0.078162	test-rmse:0.109668
[386]	train-rmse:0.078018	test-rmse:0.109677
[387]	train-

[33]	train-rmse:4.12582	test-rmse:4.13286
[34]	train-rmse:4.00377	test-rmse:4.01092
[35]	train-rmse:3.88554	test-rmse:3.89279
[36]	train-rmse:3.76967	test-rmse:3.77679
[37]	train-rmse:3.65772	test-rmse:3.66494
[38]	train-rmse:3.54901	test-rmse:3.55635
[39]	train-rmse:3.44424	test-rmse:3.45162
[40]	train-rmse:3.34251	test-rmse:3.3495
[41]	train-rmse:3.24398	test-rmse:3.25132
[42]	train-rmse:3.14787	test-rmse:3.15527
[43]	train-rmse:3.05535	test-rmse:3.06333
[44]	train-rmse:2.96527	test-rmse:2.97288
[45]	train-rmse:2.87789	test-rmse:2.88523
[46]	train-rmse:2.79285	test-rmse:2.80019
[47]	train-rmse:2.71055	test-rmse:2.71757
[48]	train-rmse:2.63138	test-rmse:2.63835
[49]	train-rmse:2.55486	test-rmse:2.56191
[50]	train-rmse:2.47965	test-rmse:2.48705
[51]	train-rmse:2.40721	test-rmse:2.41449
[52]	train-rmse:2.33615	test-rmse:2.34353
[53]	train-rmse:2.26775	test-rmse:2.27486
[54]	train-rmse:2.20097	test-rmse:2.20808
[55]	train-rmse:2.13586	test-rmse:2.14254
[56]	train-rmse:2.07186	test-rmse:2

[220]	train-rmse:0.099283	test-rmse:0.141461
[221]	train-rmse:0.099081	test-rmse:0.141319
[222]	train-rmse:0.098722	test-rmse:0.141258
[223]	train-rmse:0.098432	test-rmse:0.141229
[224]	train-rmse:0.09822	test-rmse:0.141115
[225]	train-rmse:0.098024	test-rmse:0.141299
[226]	train-rmse:0.09768	test-rmse:0.141187
[227]	train-rmse:0.097383	test-rmse:0.141047
[228]	train-rmse:0.097128	test-rmse:0.140993
[229]	train-rmse:0.096985	test-rmse:0.141003
[230]	train-rmse:0.096693	test-rmse:0.140897
[231]	train-rmse:0.09646	test-rmse:0.140975
[232]	train-rmse:0.096338	test-rmse:0.141289
[233]	train-rmse:0.096133	test-rmse:0.141205
[234]	train-rmse:0.095956	test-rmse:0.141087
[235]	train-rmse:0.09575	test-rmse:0.141195
[236]	train-rmse:0.095502	test-rmse:0.141246
[237]	train-rmse:0.095295	test-rmse:0.141195
[238]	train-rmse:0.094963	test-rmse:0.141248
[239]	train-rmse:0.094641	test-rmse:0.140982
[240]	train-rmse:0.094404	test-rmse:0.140878
[241]	train-rmse:0.094249	test-rmse:0.140981
[242]	train-rm

...

[279]	train-rmse:0.047495	test-rmse:0.129341
[280]	train-rmse:0.047317	test-rmse:0.129218
[281]	train-rmse:0.047105	test-rmse:0.129152
[282]	train-rmse:0.046942	test-rmse:0.129079
[283]	train-rmse:0.046833	test-rmse:0.129066
[284]	train-rmse:0.046605	test-rmse:0.128968
[285]	train-rmse:0.046508	test-rmse:0.12885
[286]	train-rmse:0.046349	test-rmse:0.128756
[287]	train-rmse:0.046129	test-rmse:0.128713
[288]	train-rmse:0.045828	test-rmse:0.128731
[289]	train-rmse:0.045671	test-rmse:0.128652
[290]	train-rmse:0.045479	test-rmse:0.128625
[291]	train-rmse:0.045346	test-rmse:0.128538
[292]	train-rmse:0.045253	test-rmse:0.128482
[293]	train-rmse:0.04516	test-rmse:0.128416
[294]	train-rmse:0.04505	test-rmse:0.128354
[295]	train-rmse:0.044846	test-rmse:0.128312
[296]	train-rmse:0.044662	test-rmse:0.128233
[297]	train-rmse:0.044441	test-rmse:0.128164
[298]	train-rmse:0.044325	test-rmse:0.128143
[299]	train-rmse:0.044125	test-rmse:0.128125
[300]	train-rmse:0.044007	test-rmse:0.128094
[301]	train-r

[462]	train-rmse:0.024309	test-rmse:0.126603
[463]	train-rmse:0.024214	test-rmse:0.126582
[464]	train-rmse:0.024113	test-rmse:0.126553
[465]	train-rmse:0.02394	test-rmse:0.12658
[466]	train-rmse:0.023911	test-rmse:0.126566
[467]	train-rmse:0.023834	test-rmse:0.126609
[468]	train-rmse:0.023776	test-rmse:0.126613
[469]	train-rmse:0.023705	test-rmse:0.126598
[470]	train-rmse:0.023598	test-rmse:0.126605
[471]	train-rmse:0.023477	test-rmse:0.126601
[472]	train-rmse:0.023338	test-rmse:0.126652
[473]	train-rmse:0.023279	test-rmse:0.12665
[474]	train-rmse:0.023152	test-rmse:0.126687
[475]	train-rmse:0.023096	test-rmse:0.126671
[476]	train-rmse:0.023017	test-rmse:0.126669
[477]	train-rmse:0.02293	test-rmse:0.126681
[478]	train-rmse:0.022808	test-rmse:0.12668
[479]	train-rmse:0.022735	test-rmse:0.126672
[480]	train-rmse:0.02262	test-rmse:0.126655
[481]	train-rmse:0.022558	test-rmse:0.126639
[482]	train-rmse:0.022485	test-rmse:0.126666
[483]	train-rmse:0.022408	test-rmse:0.12667
[484]	train-rmse:

[160]	train-rmse:0.137401
[161]	train-rmse:0.135283
[162]	train-rmse:0.133312
[163]	train-rmse:0.131362
[164]	train-rmse:0.12948
[165]	train-rmse:0.127612
[166]	train-rmse:0.126076
[167]	train-rmse:0.124319
[168]	train-rmse:0.122755
[169]	train-rmse:0.121273
[170]	train-rmse:0.119816
[171]	train-rmse:0.118446
[172]	train-rmse:0.116995
[173]	train-rmse:0.115562
[174]	train-rmse:0.114256
[175]	train-rmse:0.113005
[176]	train-rmse:0.111917
[177]	train-rmse:0.110778
[178]	train-rmse:0.109592
[179]	train-rmse:0.108609
[180]	train-rmse:0.107748
[181]	train-rmse:0.106596
[182]	train-rmse:0.105674
[183]	train-rmse:0.104737
[184]	train-rmse:0.103836
[185]	train-rmse:0.102896
[186]	train-rmse:0.102084
[187]	train-rmse:0.101236
[188]	train-rmse:0.100441
[189]	train-rmse:0.099742
[190]	train-rmse:0.098929
[191]	train-rmse:0.098296
[192]	train-rmse:0.097686
[193]	train-rmse:0.097133
[194]	train-rmse:0.096492
[195]	train-rmse:0.095958
[196]	train-rmse:0.095439
[197]	train-rmse:0.094812
[198]	train-r

[477]	train-rmse:0.052031
[478]	train-rmse:0.051953
[479]	train-rmse:0.051906
[480]	train-rmse:0.051823
[481]	train-rmse:0.051757
[482]	train-rmse:0.051696
[483]	train-rmse:0.051606
[484]	train-rmse:0.051515
[485]	train-rmse:0.051416
[486]	train-rmse:0.051367
[487]	train-rmse:0.051325
[488]	train-rmse:0.051224
[489]	train-rmse:0.051165
[490]	train-rmse:0.051038
[491]	train-rmse:0.050936
[492]	train-rmse:0.050868
[493]	train-rmse:0.050826
[494]	train-rmse:0.050778
[495]	train-rmse:0.05073
[496]	train-rmse:0.050691
[497]	train-rmse:0.050633
[498]	train-rmse:0.050553
[499]	train-rmse:0.050495
[500]	train-rmse:0.050405
[501]	train-rmse:0.05029
[502]	train-rmse:0.050267
[503]	train-rmse:0.050225
[504]	train-rmse:0.050111
[505]	train-rmse:0.050024
[506]	train-rmse:0.049899
[507]	train-rmse:0.049841
[508]	train-rmse:0.049773
[509]	train-rmse:0.049711
[510]	train-rmse:0.049638
[511]	train-rmse:0.049595
[512]	train-rmse:0.049507
[513]	train-rmse:0.049438
[514]	train-rmse:0.049382
[515]	train-rm

NameError: name 'submission' is not defined

In [8]:
submission_col = 'SalePrice'
submission_target = 'test_sub1.csv'
submission_name = 'input/sample_submission.csv'
submission = pd.DataFrame.from_csv(submission_name)
submission[submission_col] = np.exp(xgclassifier.predict(xg_test))
submission.to_csv(submission_target)